# Praktische vragen Deep Learning

Sabine De Vreese, Stijn Lievens en Simon De Gheselle

11 November 2025
# 1 Using timeseries_dataset_from_array and window 
Start with some (very simple) univariate data

In [2]:
import keras as ks
import numpy as np
import tensorflow as tf

In [80]:
data = np.arange(0,100)
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

1. Use `keras.utils.timeseries_dataset_from_array` to
    - chop the data into 10 non-overlapping windows of length 10
    - the resulting `Dataset` should contain individual items


In [81]:
seq_len = 10 #* Elk blokje is 10 groot
seq_stride = 10 #* Start van volgende blokje schuift 10 pos. op
batch = 1 #* Batches uitpakken voor individuele vensters

ds1 = ks.utils.timeseries_dataset_from_array(
    sequence_length=seq_len,
    sequence_stride=seq_stride,
    data=data,
    batch_size=batch,
    targets=None
).unbatch() #* Buitenste dim. weghalen.

for item in ds1:
    print(f"{item}{item.shape}")
    print("*"*50)

[0 1 2 3 4 5 6 7 8 9](10,)
**************************************************
[10 11 12 13 14 15 16 17 18 19](10,)
**************************************************
[20 21 22 23 24 25 26 27 28 29](10,)
**************************************************
[30 31 32 33 34 35 36 37 38 39](10,)
**************************************************
[40 41 42 43 44 45 46 47 48 49](10,)
**************************************************
[50 51 52 53 54 55 56 57 58 59](10,)
**************************************************
[60 61 62 63 64 65 66 67 68 69](10,)
**************************************************
[70 71 72 73 74 75 76 77 78 79](10,)
**************************************************
[80 81 82 83 84 85 86 87 88 89](10,)
**************************************************
[90 91 92 93 94 95 96 97 98 99](10,)
**************************************************


2025-12-31 16:48:59.804949: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Next, get the same result using the window function:

In [82]:
ds1_window = tf.data.Dataset.range(100).window(size=10, #* Venster met 10 elementen
                                               shift=10, #* Schuif 10 pos. op
                                               stride=1) #* getallen in het venster (0,1,2,3,...)
ds1_flatmap = ds1_window.flat_map(lambda window : window.batch(10))
for item in (ds1_flatmap):
    print(f"{item}, {item.shape}")
    print("*"*50)

[0 1 2 3 4 5 6 7 8 9], (10,)
**************************************************
[10 11 12 13 14 15 16 17 18 19], (10,)
**************************************************
[20 21 22 23 24 25 26 27 28 29], (10,)
**************************************************
[30 31 32 33 34 35 36 37 38 39], (10,)
**************************************************
[40 41 42 43 44 45 46 47 48 49], (10,)
**************************************************
[50 51 52 53 54 55 56 57 58 59], (10,)
**************************************************
[60 61 62 63 64 65 66 67 68 69], (10,)
**************************************************
[70 71 72 73 74 75 76 77 78 79], (10,)
**************************************************
[80 81 82 83 84 85 86 87 88 89], (10,)
**************************************************
[90 91 92 93 94 95 96 97 98 99], (10,)
**************************************************


2025-12-31 16:48:59.837075: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2.The same as the previous task, but the resultingDatasetshould containbatches of size 3.

In [83]:
seq_len = 10
seq_stride = 10
batch = 3

ds2 = ks.utils.timeseries_dataset_from_array(
    targets=None,
    data=data,
    sequence_length=seq_len,
    sequence_stride=seq_stride,
    batch_size=batch,
)

for item in ds2:
    print(f"{item}{item.shape}")
    print("*"*50)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]](3, 10)
**************************************************
[[30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]](3, 10)
**************************************************
[[60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]](3, 10)
**************************************************
[[90 91 92 93 94 95 96 97 98 99]](1, 10)
**************************************************


2025-12-31 16:48:59.880421: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3. The same as the previous task, but now the sequences should overlap by 5 elements. Hence, the desired output is

In [84]:
seq_stride = 5
seq_len = 10
batch = 3
ds3 = ks.utils.timeseries_dataset_from_array(
    sequence_stride=seq_stride,
    sequence_length=seq_len,
    batch_size=batch,
    data=data,
    targets=None
)

for item in ds3:
    print(f"{item}{item.shape}")
    print("*"*50)

[[ 0  1  2  3  4  5  6  7  8  9]
 [ 5  6  7  8  9 10 11 12 13 14]
 [10 11 12 13 14 15 16 17 18 19]](3, 10)
**************************************************
[[15 16 17 18 19 20 21 22 23 24]
 [20 21 22 23 24 25 26 27 28 29]
 [25 26 27 28 29 30 31 32 33 34]](3, 10)
**************************************************
[[30 31 32 33 34 35 36 37 38 39]
 [35 36 37 38 39 40 41 42 43 44]
 [40 41 42 43 44 45 46 47 48 49]](3, 10)
**************************************************
[[45 46 47 48 49 50 51 52 53 54]
 [50 51 52 53 54 55 56 57 58 59]
 [55 56 57 58 59 60 61 62 63 64]](3, 10)
**************************************************
[[60 61 62 63 64 65 66 67 68 69]
 [65 66 67 68 69 70 71 72 73 74]
 [70 71 72 73 74 75 76 77 78 79]](3, 10)
**************************************************
[[75 76 77 78 79 80 81 82 83 84]
 [80 81 82 83 84 85 86 87 88 89]
 [85 86 87 88 89 90 91 92 93 94]](3, 10)
**************************************************
[[90 91 92 93 94 95 96 97 98 99]](1, 10)
***********

2025-12-31 16:48:59.913566: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


4. Add targets. The target for each sequence of length 10 should be the ele-ment immediately following the sequence. No batching is required. Hence,the desired output for:

In [85]:
seq_stride = 5
seq_len = 10
batch = None
ds4 = ks.utils.timeseries_dataset_from_array(
    sequence_length=seq_len,
    sequence_stride=seq_stride,
    batch_size=batch,
    data=data,
    targets=data[10:]
)
for item, label in ds4:
    print(f"{item}maps to {label} and has shape{item.shape}")
    print("*"*50)

[0 1 2 3 4 5 6 7 8 9]maps to 10 and has shape(10,)
**************************************************
[ 5  6  7  8  9 10 11 12 13 14]maps to 15 and has shape(10,)
**************************************************
[10 11 12 13 14 15 16 17 18 19]maps to 20 and has shape(10,)
**************************************************
[15 16 17 18 19 20 21 22 23 24]maps to 25 and has shape(10,)
**************************************************
[20 21 22 23 24 25 26 27 28 29]maps to 30 and has shape(10,)
**************************************************
[25 26 27 28 29 30 31 32 33 34]maps to 35 and has shape(10,)
**************************************************
[30 31 32 33 34 35 36 37 38 39]maps to 40 and has shape(10,)
**************************************************
[35 36 37 38 39 40 41 42 43 44]maps to 45 and has shape(10,)
**************************************************
[40 41 42 43 44 45 46 47 48 49]maps to 50 and has shape(10,)
**************************************************
[4

2025-12-31 16:48:59.956141: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


5. From here on, only use thewindowmethod.Map sequences of length 5 to a single sequence of length 2 that containsthe next two elements. Sequences should overlap by 4 elements. Desired output for 

    ```python
    [0 1 2 3 4] maps to [5 6] and has shape (5,)**************************************************[1 2 3 4 5] maps to [6 7] and has shape (5,)**************************************************
    .....
    [93 94 95 96 97] maps to [98 99] and has shape (5,)**************************************************
    ```

In [115]:
size = 7
stide = 1
shift = 1
batch = 7
ds5_window = tf.data.Dataset.range(100).window(
    shift=shift,
    size=size,
    stride=stide,
    drop_remainder=True
).flat_map(
    lambda window: window.batch(7)
).map(
    lambda window: (window[:5],window[5:])
)

for item, label in ds5_window:
    print(f"{item} maps to {label} and has shape {item.shape}")
    print("*"*50)

[0 1 2 3 4] maps to [5 6] and has shape (5,)
**************************************************
[1 2 3 4 5] maps to [6 7] and has shape (5,)
**************************************************
[2 3 4 5 6] maps to [7 8] and has shape (5,)
**************************************************
[3 4 5 6 7] maps to [8 9] and has shape (5,)
**************************************************
[4 5 6 7 8] maps to [ 9 10] and has shape (5,)
**************************************************
[5 6 7 8 9] maps to [10 11] and has shape (5,)
**************************************************
[ 6  7  8  9 10] maps to [11 12] and has shape (5,)
**************************************************
[ 7  8  9 10 11] maps to [12 13] and has shape (5,)
**************************************************
[ 8  9 10 11 12] maps to [13 14] and has shape (5,)
**************************************************
[ 9 10 11 12 13] maps to [14 15] and has shape (5,)
**************************************************
[10 11 1

2025-12-31 17:00:52.888057: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


6. Add shuffling `(buffer_size=10_000 and seed=42)` to the previous task.Also batch in batches of 10. The code

In [130]:
ds6_window = ds5_window.shuffle(
    buffer_size=10_000,
    seed=42
).batch(10)

for item, label in ds6_window:
    print(f"{item} maps to {label} and has shape{item.shape}")
    print("*"*50)

[[15 16 17 18 19]
 [72 73 74 75 76]
 [10 11 12 13 14]
 [67 68 69 70 71]
 [ 3  4  5  6  7]
 [56 57 58 59 60]
 [46 47 48 49 50]
 [77 78 79 80 81]
 [76 77 78 79 80]
 [49 50 51 52 53]] maps to [[20 21]
 [77 78]
 [15 16]
 [72 73]
 [ 8  9]
 [61 62]
 [51 52]
 [82 83]
 [81 82]
 [54 55]] and has shape(10, 5)
**************************************************
[[54 55 56 57 58]
 [63 64 65 66 67]
 [19 20 21 22 23]
 [ 5  6  7  8  9]
 [12 13 14 15 16]
 [33 34 35 36 37]
 [71 72 73 74 75]
 [87 88 89 90 91]
 [14 15 16 17 18]
 [48 49 50 51 52]] maps to [[59 60]
 [68 69]
 [24 25]
 [10 11]
 [17 18]
 [38 39]
 [76 77]
 [92 93]
 [19 20]
 [53 54]] and has shape(10, 5)
**************************************************
[[20 21 22 23 24]
 [89 90 91 92 93]
 [42 43 44 45 46]
 [17 18 19 20 21]
 [32 33 34 35 36]
 [58 59 60 61 62]
 [68 69 70 71 72]
 [59 60 61 62 63]
 [93 94 95 96 97]
 [91 92 93 94 95]] maps to [[25 26]
 [94 95]
 [47 48]
 [22 23]
 [37 38]
 [63 64]
 [73 74]
 [64 65]
 [98 99]
 [96 97]] and has shape(10

2025-12-31 17:10:09.742199: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
